In [ ]:
# Imports

In [10]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage
from sklearn.feature_extraction.text import TfidfVectorizer
## nltk imports
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
# ! pip install xlrd
# ! pip install openpyxl
## lda 
from gensim import corpora
import gensim

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## added path here; may need to change to location of 
## where your dropbox folder is
PATH_TO_DROPBOX = "../../../qss20_finalproj_rawdata/textasdata/"
PATH_TO_DROPBOX


'../../../qss20_finalproj_rawdata/textasdata/'

# Step 1: read in combined addendums and disclosure 

In [11]:
# load data
addendum = pd.read_csv(PATH_TO_DROPBOX + "FOIA_2021-F-05932_raw_data_combined.csv")
disclosure = pd.read_csv (PATH_TO_DROPBOX + "H-2A_Disclosure_Data_FY_combined.csv")

/Users/rebeccajohnson/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (17,18,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
addendum.SECTION_NAME.value_counts()

Job Duties - null                                                                            3596
Job Requirements                                                                             2357
Job Duties                                                                                   1937
Job Duties - undefined                                                                       1734
Inbound/Outbound Transportation - null                                                       1641
                                                                                             ... 
Job Requirements - B.6. Additional Information Regarding Job Qualifications/Requirements.       1
Job Requirements - B.6 Additional Information Regarding Job Qualifications/ Requirements        1
Job Duties - Work Rules _Addendum 3                                                             1
Pay Deductions - A.8a Additional A.2a Workers needed (f)                                        1
Pay Deductions - A.8

In [12]:
# Dropping duplicates in addenda

## perhaps we want to create a priority order for which section we should keep if the
## same data is inputted in multiple sections?

len(addendum)
addendum = addendum.drop_duplicates(["CASE_NUMBER", "SECTION_DETAILS"])
len(addendum)

102317

99620

In [79]:
## collapsing identical case/sections into one listing each

# addendum.sort_values(["CASE_NUMBER", "SECTION_NUMBER"])

# addendum["SECTION_DETAILS"].to_string()

addendum = addendum.groupby(['CASE_NUMBER'])['SECTION_DETAILS'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', '').reset_index()



<ipython-input-79-6dac8fc188fc>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  addendum = addendum.groupby(['CASE_NUMBER'])['SECTION_DETAILS'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', '').reset_index()


In [ ]:
## Above code modified from fifth answer at https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
## because original groupby was including a bunch of linebreaks

In [8]:
# Rename Column
addendum = addendum.rename(columns = {"SECTION_DETAILS":"JOB_DESCRIPTION"})

In [9]:
#Remove Job Listings that Contain Spanish Words
spanish_words = ['compensación',
 'ocupación',
 'pago',
 'transporte',
 'reglas',
 'contrato',
 'horastrabajadores',
 'parte',
 'tarifas',
 'cuartos',
 'trabajo',
 'registros',
 'adicionales',
 'comidas',
 'empleo',
 'terminación',
 'vivienda',
 'empleador',
 'adicionales',
 'frecuencia',
 'producción',
 'liquidación',
 'comida',
 'herramientas',
 'prueba',
 'diaria',
 'tarifas',
 'parado',
 'adicionales',
 'cobertura',
 'proporcionados',
 'parte',
 'abandono',
 'deducciones',
 'causa',
 'garantías',
 'divulgación',
 'comidas',
 'política',
 'disciplina',
 'despidos',
 'derechos',
 'imposibilidad',
 'detalles',
 'divulgación',
 'bloqueo',
 'abandono',
 'ganancias',
 'cobertura',
 'pedidos',
 'pago',
 'oportunidad',
 'trabajo']

addendum = addendum[~addendum.JOB_DESCRIPTION.str.contains('|'.join(spanish_words))]
print(len(addendum))


11763


In [53]:
## Text Processing
def processtext(row, colname, stopword_list, min_token_length = 4):
    
    ## get string form of listing
    string_of_col = str(row[colname]).lower()
    try:
        ## remove stopwords 
        remove_stop = [word for word in wordpunct_tokenize(string_of_col)
                      if word not in stopword_list]
        processed_string = " ".join([porter.stem(i) 
                        for i in remove_stop if 
                        i.isalpha() and len(i) >= min_token_length])
        return(processed_string)
    except:
        processed_string = "" # to handle data errors where not actually text
        return(processed_string)
    
other_stopwords = ["after", "before", "employer", "employ", "job", "work", "worker"]

list_stopwords = stopwords.words("english")

stopwords_complete = list_stopwords + other_stopwords
    
porter = PorterStemmer()



In [54]:
stopwords_complete = list_stopwords + other_stopwords

addendum['text_preprocess'] = addendum.apply(processtext,
                             axis = 1,
                             args = ["JOB_DESCRIPTION", stopwords_complete])
addendum.head()

,CASE_NUMBER,JOB_DESCRIPTION,text_preprocess
0,H-300-19274-066154,Operating large farm tractors and equipment as...,oper larg farm tractor equip pleas contact poi...
1,H-300-19274-066174,Employer will furnish free and convenient cook...,furnish free conveni cooki three month experi ...
2,H-300-19274-066180,INBOUND 1) Advance the worker the required tra...,inbound advanc requir tran provid cost whether...
3,H-300-19274-066199,"For workers who are provided housing, transpor...",worker provid hous transport previous advanc t...
4,H-300-19274-066237,The inbound transportation will be reimbursed ...,inbound transport reimburs


In [55]:
#Make Sure all the rows under EMPLOYER_STATE has two digit state code
disclosure.EMPLOYER_STATE=disclosure.EMPLOYER_STATE.astype("string")
disclosure=disclosure[~disclosure.EMPLOYER_STATE.isnull()]
assert all(disclosure.EMPLOYER_STATE.str.len()==2), "All the states in debar are not two-digits after the cleaning"

# Label TRLA Catchment State VS Remaining States
disclosure["TRLA"] = np.where((disclosure["EMPLOYER_STATE"] == "TX") | 
                                          (disclosure["EMPLOYER_STATE"] == "MS")|(disclosure["EMPLOYER_STATE"] == "LA")|
                              (disclosure["EMPLOYER_STATE"] == "KY")|(disclosure["EMPLOYER_STATE"] == "AL")|(disclosure["EMPLOYER_STATE"] == 
                                                                                                          "TN"), True, False)
disclosure.TRLA.value_counts()

False    11966
True      4801
Name: TRLA, dtype: int64

In [56]:
#Merging

In [57]:
# merging testing code (fully applicable after data cleaning and the necessary steps above)
## left join  
merged_data_left = pd.merge(disclosure,
                             addendum,
                             left_on = 'CASE_NUMBER',
                             right_on = 'CASE_NUMBER',
                             how = "left",
                             indicator = "case_merge_status")
merged_data_left.case_merge_status.value_counts()
merged_data_left

both          11742
left_only      5025
right_only        0
Name: case_merge_status, dtype: int64

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,TRLA,JOB_DESCRIPTION,text_preprocess,case_merge_status
0,H-300-20265-835437,Determination Issued - Certification,2020/9/24,2020/9/30,Individual Employer,Y,Seasonal,N,Benito Martinez Jr.,NaN,...,$12.68,$55.00,14073199505,benitomartinez863@gmail.com,NaN,0.0,False,NaN,NaN,left_only
1,H-300-20260-827678,Determination Issued - Certification,2020/9/16,2020/9/24,Association - Joint Employer,N,Seasonal,N,WESTERN RANGE ASSOCIATION,NaN,...,$12.68,$55.00,12085952226,applicants@westernrange.net,NaN,0.0,False,Debe tener 3 meses de experiencia con 800 - 10...,debe tener mese experiencia incom transport su...,both
2,H-300-20260-827308,Determination Issued - Certification,2020/9/16,2020/9/24,Association - Agent,N,Seasonal,N,Allen Ranches LLC,NaN,...,$12.68,$55.00,13074722105,info@mpaswy.com,NaN,0.0,False,An employee may be terminated for just cause. ...,employe termin caus worker requir live ranch t...,both
3,H-300-20260-826330,Determination Issued - Certification,2020/9/18,2020/9/30,Individual Employer,N,Seasonal,N,Steven Edge,NaN,...,$12.68,$55.00,18433407334,sdedge@sccoast.net,NaN,0.0,False,NaN,NaN,left_only
4,H-300-20259-823987,Determination Issued - Withdrawn,2020/9/18,2020/9/29,Individual Employer,N,Seasonal,N,Floyd Frey,NaN,...,$12.68,$55.00,14062650953,floydfrey@yahoo.com,NaN,0.0,False,"Kitchen facilities, utilities and utensils wil...",kitchen facil util utensil depend situat trans...,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16762,H-300-20351-963307,Determination Issued - Certification,2020/12/17,2020/12/31,Individual Employer,N,Seasonal,N,James L Schneller,NaN,...,$12.68,$55.00,12709914373,jims4463@aol.com,NaN,0.0,True,Operating tools and equipment as applicable an...,oper tool equip applic furnish free conveni co...,both
16763,H-300-20351-963399,Determination Issued - Certification,2020/12/17,2020/12/31,Individual Employer,N,Seasonal,N,Stephen G Myers,NaN,...,$12.68,$55.00,12708780028,vjpm76@gmail.com,https://kentucky.gov/employment/Pages/default....,1.0,True,Workers who qualify for inbound and/or outboun...,worker qualifi inbound outbound,both
16764,H-300-20351-964097,Determination Issued - Certification,2020/12/22,2020/12/31,Individual Employer,N,Seasonal,N,Andy Povey,Andy Povey Farms,...,$12.68,$55.00,NaN,H-2AJobs@snakeriverfarmers.org,https://idahoworks.gov/ada/r/job_seeker,0.0,False,The worker may be offered work seven days a we...,offer seven day deduct except requir extent fo...,both
16765,H-300-20351-965435,Determination Issued - Certification,2020/12/22,2020/12/31,Individual Employer,N,Seasonal,N,"Silver Creek Seed, LLC",NaN,...,$12.68,$55.00,NaN,H-2AJobs@snakeriverfarmers.org,https://idahoworks.gov/ada/r/job_seeker,0.0,False,Inbound transportation and subsistence fees wi...,inbound transport subsist fee duti requir spli...,both


In [58]:
# ## look at one that doesnt have job listing
merged_left_check = merged_data_left[['case_merge_status', 'CASE_NUMBER',"JOB_DESCRIPTION","TRLA","EMPLOYER_STATE"]]
merged_left_check=merged_left_check[merged_left_check.case_merge_status=="left_only"]
merged_left_check.TRLA.value_counts()
disclosure.TRLA.value_counts()
#percentage of non TRLA states is higher
#compare wages? is it that there's lower wage rate so there's no listing?


False    3708
True     1317
Name: TRLA, dtype: int64

False    11966
True      4801
Name: TRLA, dtype: int64

In [59]:
## inner join  
merged_data = pd.merge(addendum,
                             disclosure,
                             left_on = 'CASE_NUMBER',
                             right_on = 'CASE_NUMBER',
                             how = "inner",
                             indicator = "case_merge_status")
merged_data.case_merge_status.value_counts()


both          11742
left_only         0
right_only        0
Name: case_merge_status, dtype: int64

In [60]:
#Subset Data to TRLA catchment

merged_TRLA=merged_data[merged_data.TRLA==True]
merged_TRLA
#Subset Data to remaining states
merged_nonTRLA=merged_data[merged_data.TRLA==False]
merged_nonTRLA

,CASE_NUMBER,JOB_DESCRIPTION,text_preprocess,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,...,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,TRLA,case_merge_status
1,H-300-19274-066174,Employer will furnish free and convenient cook...,furnish free conveni cooki three month experi ...,Determination Issued - Withdrawn,2019/10/1,2019/10/4,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,15124752571,foreignlabor@twc.state.tx.us,NaN,0.0,True,both
3,H-300-19274-066199,"For workers who are provided housing, transpor...",worker provid hous transport previous advanc t...,Determination Issued - Certification,2019/10/18,2019/11/12,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,14096589722,zdishman83@yahoo.com,NaN,1.0,True,both
6,H-300-19274-066444,Employer will pay for or reimburse the worker ...,reimburs actual employ offer sole disc,Determination Issued - Certification,2019/10/1,2019/11/18,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,18309005090,NaN,www.workintexas.com,1.0,True,both
7,H-300-19275-067016,The employer will reimburse the worker for the...,reimburs request hour nthe demand competit bus...,Determination Issued - Certification,2019/10/4,2019/11/15,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,13372244135,Eugene.Zaunbrecher@gmail.com,NaN,0.0,True,both
8,H-300-19275-067031,Employer will provide workers with transportat...,provid worker transportati reimburs demand com...,Determination Issued - Certification,2019/10/4,2019/11/21,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,13377899868,SheltonMarcus.Miller@gmail.com,NaN,0.0,True,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11656,H-300-20343-941481,Bonuses and/or wages that are higher than the ...,bonus wage higher requir avail hour tool suppl...,Determination Issued - Certification,2020/12/8,2020/12/31,Association - Agent,N,Seasonal,N,...,N,NaN,$12.68,$55.00,13074722105,info@mpaswy.com,NaN,0.0,True,both
11660,H-300-20343-941606,16.\tAny worker who physically threatens anoth...,tani physic threaten anoth although intend com...,Determination Issued - Certification,2020/12/9,2020/12/28,Association - Agent,N,Seasonal,N,...,N,NaN,$12.68,$55.00,12707267768,NaN,https://focuscareer.ky.gov/careerexplorer/jobs...,0.0,True,both
11672,H-300-20343-942095,Employer may reward exceptional work with mone...,reward except monet,Determination Issued - Certification,2020/12/11,2020/12/21,Individual Employer,N,Seasonal,N,...,N,NaN,$12.68,$55.00,13345223238,ckwillob@gmail.com,NaN,0.0,True,both
11730,H-300-20351-963307,Operating tools and equipment as applicable an...,oper tool equip applic furnish free conveni co...,Determination Issued - Certification,2020/12/17,2020/12/31,Individual Employer,N,Seasonal,N,...,N,NaN,$12.68,$55.00,12709914373,jims4463@aol.com,NaN,0.0,True,both


,CASE_NUMBER,JOB_DESCRIPTION,text_preprocess,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,...,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,TRLA,case_merge_status
0,H-300-19274-066154,Operating large farm tractors and equipment as...,oper larg farm tractor equip pleas contact poi...,Determination Issued - Withdrawn,2019/10/1,2019/10/23,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,16057734212,taige.tople@state.sd.us,NaN,0.0,False,both
2,H-300-19274-066180,INBOUND 1) Advance the worker the required tra...,inbound advanc requir tran provid cost whether...,Determination Issued - Certification,2019/10/2,2019/10/18,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,18017545802,jcjepperson@gmail.com,NaN,0.0,False,both
4,H-300-19274-066237,The inbound transportation will be reimbursed ...,inbound transport reimburs,Determination Issued - Certification (Expired),2019/10/18,2019/12/2,Individual Employer,Y,Seasonal,N,...,N,NaN,$12.46,$55.00,18634640496,sbharvestinginc@gmail.com,NaN,0.0,False,both
5,H-300-19274-066365,"Alternatively, the employer may, at his or her...",altern,Determination Issued - Withdrawn,2019/10/4,2019/10/4,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,17017434545,lynnb@restel.net,NaN,0.0,False,both
9,H-300-19275-067184,"Alternatively, the employer may, at his or her...",altern,Determination Issued - Certification (Expired),2019/10/11,2019/11/15,Individual Employer,N,Seasonal,N,...,N,NaN,$12.46,$55.00,16058552933,kirkro@ndsupernet.com,NaN,0.0,False,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,H-300-20351-965074,Employer will furnish free and convenient cook...,furnish free conveni cooki employ compens insu...,Determination Issued - Certification,2020/12/17,2020/12/23,Individual Employer,N,Seasonal,N,...,N,NaN,$12.68,$55.00,19705391015,frankag2011@gmail.com,NaN,0.0,False,both
11738,H-300-20351-965273,The number of workers shown is the aggregate n...,number worker shown aggreg equip oper field op...,Determination Issued - Certification,2020/12/17,2020/12/22,Individual Employer,N,Seasonal,N,...,N,NaN,$12.68,$55.00,18433358335,NaN,https://seasonaljobs.dol.gov/,0.0,False,both
11739,H-300-20351-965435,Inbound transportation and subsistence fees wi...,inbound transport subsist fee duti requir spli...,Determination Issued - Certification,2020/12/22,2020/12/31,Individual Employer,N,Seasonal,N,...,Y,$12.68,$12.68,$55.00,NaN,H-2AJobs@snakeriverfarmers.org,https://idahoworks.gov/ada/r/job_seeker,0.0,False,both
11740,H-300-20356-972294,"16.\tWorker may not falsify identification, pe...",tworker falsifi identif hous rule tthi hous te...,Determination Issued - Certification,2020/12/21,2020/12/29,Joint Employer,N,Seasonal,N,...,N,NaN,$12.68,$55.00,18644720207,NaN,https://seasonaljobs.dol.gov/,0.0,False,both


In [61]:
## topic modeling (Top Words In Combined Data)
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(merged_data['text_preprocess'])
top_combined = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())
top_combined.sum(axis=0).sort_values(ascending=False).head(10)

worker       972.281521
provid       748.928728
transport    612.770280
complet      532.019871
termin       470.511803
hous         463.696189
advanc       450.541257
requir       444.695451
subsist      384.799561
inbound      382.561694
dtype: float64

In [62]:
## topic modeling (Top Words In TRLA)
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(merged_nonTRLA['text_preprocess'])
top_combined_TRLA = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())
top_combined_TRLA.sum(axis=0).sort_values(ascending=False).head(10)

worker       680.029314
provid       590.677363
transport    478.561728
termin       416.962515
advanc       405.318280
requir       386.504611
hous         342.223064
subsist      341.129475
inbound      338.848663
offer        263.010323
dtype: float64

In [63]:
## topic modeling (Top Words In Non-TRLA)
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(merged_TRLA['text_preprocess'])
top_combined_nonTRLA = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())
top_combined_nonTRLA.sum(axis=0).sort_values(ascending=False).head(10)

worker      365.497361
complet     317.096067
time        254.771197
appli       242.341893
domest      239.945642
tani        239.399262
threaten    239.399262
anoth       239.383324
legal       239.098346
physic      238.843095
dtype: float64

In [64]:
## topic modeling (TRLA states)

merged_TRLA = merged_TRLA[merged_TRLA.text_preprocess != ""].copy()

tokenized_text_TRLA = [wordpunct_tokenize(one_text) for one_text in 
                                      merged_TRLA.text_preprocess]

## preprocess and estimate topicmod
### create dictionary
text_proc_dict_TRLA = corpora.Dictionary(tokenized_text_TRLA)
### filter dictionary- using 2% as bounds
text_proc_dict_TRLA.filter_extremes(no_below = round(merged_TRLA.shape[0]*0.02),
                             no_above = round(merged_TRLA.shape[0]*0.98))

### create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict_TRLA.doc2bow(one_text) 
                   for one_text in tokenized_text_TRLA]

tfidf = models.TfidfModel(corpus_fromdict_proc)
corpus_tfidf = tfidf[corpus_fromdict_proc]

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=text_proc_dict_TRLA, passes=6,
                                        per_word_topics = True, random_state = 91988)

topics = lda_model_tfidf.print_topics(num_words = 10)
for topic in topics:
    print(topic)

(0, '0.056*"tobacco" + 0.056*"assign" + 0.053*"nworker" + 0.050*"begin" + 0.049*"pull" + 0.047*"sucker" + 0.033*"must" + 0.033*"worker" + 0.027*"time" + 0.026*"week"')
(1, '0.051*"transport" + 0.049*"contact" + 0.034*"shall" + 0.032*"point" + 0.032*"provid" + 0.030*"cooki" + 0.030*"conveni" + 0.030*"reason" + 0.029*"furnish" + 0.028*"repair"')
(2, '0.050*"mechan" + 0.050*"vari" + 0.038*"avail" + 0.029*"rule" + 0.027*"time" + 0.021*"hous" + 0.020*"provid" + 0.020*"hour" + 0.019*"employe" + 0.019*"advanc"')
(3, '0.066*"complet" + 0.062*"appli" + 0.061*"anoth" + 0.061*"tani" + 0.061*"threaten" + 0.061*"legal" + 0.060*"physic" + 0.060*"domest" + 0.060*"although" + 0.058*"list"')


In [74]:
## topic modeling (nonTRLA states)

merged_nonTRLA = merged_nonTRLA[merged_nonTRLA.text_preprocess != ""].copy()

tokenized_text_nonTRLA = [wordpunct_tokenize(one_text) for one_text in 
                                      merged_nonTRLA.text_preprocess]

## preprocess and estimate topicmod
### create dictionary
text_proc_dict_nonTRLA = corpora.Dictionary(tokenized_text_nonTRLA)
### filter dictionary- using 2% as bounds
text_proc_dict_nonTRLA.filter_extremes(no_below = round(merged_nonTRLA.shape[0]*0.02),
                             no_above = round(merged_nonTRLA.shape[0]*0.98))

### create corpus from dictionary
corpus_fromdict_proc_non = [text_proc_dict_nonTRLA.doc2bow(one_text) 
                   for one_text in tokenized_text_nonTRLA]

tfidf_non = models.TfidfModel(corpus_fromdict_proc_non)
corpus_tfidf_non = tfidf[corpus_fromdict_proc_non]

lda_model_tfidf_non = gensim.models.LdaMulticore(corpus_tfidf_non, num_topics=4, id2word=text_proc_dict_nonTRLA, passes=6,
                                        per_word_topics = True, random_state = 91988)

topics_nontrla = lda_model_tfidf_non.print_topics(num_words = 10)
for topic in topics_nontrla:
    print(topic)

(0, '0.097*"complet" + 0.093*"provid" + 0.056*"worker" + 0.037*"place" + 0.035*"contract" + 0.030*"offer" + 0.030*"harvest" + 0.027*"nthe" + 0.027*"advanc" + 0.025*"number"')
(1, '0.047*"worker" + 0.038*"hous" + 0.035*"provid" + 0.025*"cost" + 0.025*"tworker" + 0.023*"offer" + 0.023*"follow" + 0.023*"harvest" + 0.022*"rule" + 0.021*"hour"')
(2, '0.109*"inbound" + 0.061*"requir" + 0.058*"day" + 0.052*"offer" + 0.052*"need" + 0.046*"instruct" + 0.045*"outbound" + 0.044*"train" + 0.044*"transport" + 0.043*"subsist"')
(3, '0.069*"transport" + 0.047*"suppli" + 0.046*"tool" + 0.040*"equip" + 0.038*"wage" + 0.035*"bonus" + 0.032*"higher" + 0.028*"employe" + 0.025*"phone" + 0.021*"commun"')


In [75]:
## topic modeling (whole document)
 
merged_data = merged_data[merged_data.text_preprocess != ""].copy()

tokenized_text_all = [wordpunct_tokenize(one_text) for one_text in 
                                      merged_data.text_preprocess]

## preprocess and estimate topicmod
### create dictionary
text_proc_dict_all = corpora.Dictionary(tokenized_text_all)
### filter dictionary- using 2% as bounds
text_proc_dict_all.filter_extremes(no_below = round(merged_data.shape[0]*0.02),
                             no_above = round(merged_data.shape[0]*0.98))

### create corpus from dictionary
corpus_fromdict_proc_all = [text_proc_dict_all.doc2bow(one_text) 
                   for one_text in tokenized_text_all]

tfidf_all = models.TfidfModel(corpus_fromdict_proc_all)
corpus_tfidf_all = tfidf[corpus_fromdict_proc_all]

lda_model_tfidf_all = gensim.models.LdaMulticore(corpus_tfidf_all, num_topics=4, id2word=text_proc_dict_all, passes=6,
                                        per_word_topics = True, random_state = 91988)

topics_all = lda_model_tfidf_all.print_topics(num_words = 10)
for topic in topics_all:
    print(topic)

(0, '0.076*"worker" + 0.058*"transport" + 0.040*"subsist" + 0.036*"requir" + 0.036*"offer" + 0.032*"must" + 0.028*"commun" + 0.027*"deduct" + 0.026*"train" + 0.026*"incom"')
(1, '0.136*"worker" + 0.047*"employe" + 0.044*"although" + 0.041*"physic" + 0.038*"legal" + 0.038*"suppli" + 0.038*"list" + 0.037*"threaten" + 0.037*"reserv" + 0.037*"anoth"')
(2, '0.071*"rule" + 0.052*"worker" + 0.048*"time" + 0.047*"provid" + 0.037*"cont" + 0.033*"hous" + 0.031*"state" + 0.031*"specif" + 0.030*"contract" + 0.030*"vari"')
(3, '0.078*"transport" + 0.044*"worker" + 0.040*"applic" + 0.036*"provid" + 0.036*"cost" + 0.035*"contact" + 0.031*"employ" + 0.028*"furnish" + 0.026*"conveni" + 0.024*"cooki"')


In [73]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


topic = lda_model_tfidf.show_topics(formatted=False)
data_flat = [w for w_list in tokenized_text_TRLA for w in w_list]
tfid= TfidfVectorizer(data_flat)

out = []
for i, topic in topic:
    for word, weight in topic:
        out.append([word, i , weight, tfid[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count of Each Topic Keyword', fontsize=22, y=1.05)    
plt.show()


/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass input=['furnish', 'free', 'conveni', 'cooki', 'three', 'month', 'experi', 'refer', 'live', 'laundri', 'facil', 'avail', 'hous', 'employ', 'compens', 'insu', 'shall', 'provid', 'transport', 'reason', 'repair', 'cost', 'damag', 'deliberat', 'oper', 'larg', 'farm', 'tractor', 'equip', 'pleas', 'contact', 'point', 'contact', 'provid', 'transport', 'airpla', 'worker', 'provid', 'hous', 'transport', 'previous', 'advanc', 'transpo', 'provid', 'cook', 'kitchen', 'facil', 'reimburs', 'actual', 'employ', 'offer', 'sole', 'disc', 'reimburs', 'request', 'hour', 'nthe', 'demand', 'competit', 'busin', 'reimburs', 'provid', 'worker', 'transportati', 'reimburs', 'demand', 'competit', 'busin', 'provid', 'worker', 'transportati', 'previous', 'advanc', 'transpo', 'worker', 'provid', 'hous', 'transport', 'time', 'vari', 'mechan', 'avail', 'transport', 'relat', 'subsist', 'worker', 'provid', 'hous', 'transport', 'pre

TypeError: 'TfidfVectorizer' object is not subscriptable

In [ ]:
#^above visualization code adopted from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/